In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

In [5]:
data = pd.read_csv('data_0514_20260119.csv',encoding='CP949',dtype=str)

In [ ]:
start = "2025-06-01"
end = "2026-01-30"

Kospi_prices = yf.download('^KS11',start=start,end=end)['Close']['^KS11']

weekly_return1 = Kospi_prices.pct_change(periods=5)[5:] * 100 

weekly_return1.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [10]:
def RS_week(ticker):

    try:
        ETF_prices = yf.download(f'{ticker}.KS',start=start,end=end)['Close'][f'{ticker}.KS']
        
        weekly_return2 = ETF_prices.pct_change(periods=5)[5:] * 100

        weekly_return2.dropna(inplace=True)

        RS = weekly_return2 - weekly_return1

        return RS,ticker
    
    except:
        print(ticker)

In [ ]:
corp_code = data['종목코드']
corp_list = []
today_str = datetime.now().strftime('%Y-%m-%d')

for i in corp_code:
    st,ticker = RS_week(i)
    if st[-1]>=0:
        corp_list.append(ticker)

df = pd.DataFrame(corp_list,columns=['corp_list'])


# st = RS_week('002350')

# st.plot()

# plt.axhline(0, color='red', linestyle='--', linewidth=1);

# plt.show();

[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_39168\2984999616.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if st[-1]>=0:
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_39168\2984999616.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if st[-1]>=0:
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_39168\2984999616.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

ModuleNotFoundError: No module named 'openpyxl'

In [14]:
file_name = f'corp_list_{today_str}.xlsx'

df.to_csv(file_name, index=False)
print(f"✅ {file_name} 파일 생성 및 저장 완료!")

✅ corp_list_2026-01-20.xlsx 파일 생성 및 저장 완료!
